In [31]:
import tensorflow as tf 
from keras.datasets import fashion_mnist as fm

(X_train, y_train), (X_test, y_test) = fm.load_data()   
X_train, y_train = X_train[:55000], y_train[:55000]  # Use only the first 60,000 samples
X_valid,y_valid = X_train[-5000:], y_train[-5000:]

In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

tf.random.set_seed(42)
from keras.layers import Normalization
norm_layer = Normalization(input_shape=(X_train.shape[1:]))

model = Sequential([
    norm_layer,
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)
])
o = Adam(learning_rate=1e-3)
model.compile(optimizer=o,loss="mse",metrics=["RootMeanSquaredError"])

norm_layer.adapt(X_train)

history = model.fit(X_train,y_train,epochs = 20, validation_data=(X_valid,y_valid))

mse_test, rmse_test = model.evaluate(X_test, y_test)

X_new = X_test[:3]
ypred = model.predict(X_new)

/opt/anaconda3/envs/deepL/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - RootMeanSquaredError: 3.0687 - loss: 9.5048 - val_RootMeanSquaredError: 2.8646 - val_loss: 8.2057
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - RootMeanSquaredError: 2.8842 - loss: 8.3188 - val_RootMeanSquaredError: 2.8625 - val_loss: 8.1940
Epoch 3/20
1406/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 2.8796 - loss: 8.2924

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 28, 28)         │            57 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28, 50)         │         1,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 28, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,862 (77.59 KB)

 Trainable params: 6,601 (25.79 KB)

 Non-trainable params: 57 (232.00 B)

 Optimizer params: 13,204 (51.58 KB)

In [33]:
# Complex models using functional API
from keras.datasets import california_housing
from keras.models import Model
from keras.layers import Input, Dense, Normalization, Concatenate

(X_train, y_train), (X_test, y_test) = california_housing.load_data()
X_train, y_train = X_train[:14000], y_train[:14000]
X_valid, y_valid = X_train[14000:], y_train[14000:]

input_  = Input(shape=(X_train.shape[1:]))

norm_layer = Normalization()
normalized = norm_layer(input_)
norm_layer.adapt(X_train)

hidden1 = Dense(30, activation='relu')(normalized)
hidden2 = Dense(30, activation='relu')(hidden1)

concat = Concatenate()([input_, hidden2]) # we combine the two branches in order to have more knowledge for the last layer 

output = Dense(1)(concat)

model = Model(inputs =input_, outputs = output)


In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 8)         │         17 │ input_layer_1[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 30)        │        270 │ normalization_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 30)        │        930 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 38)        │          0 │ input_layer_1[0]… │
│ (Concatenate)       │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         39 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,256 (4.91 KB)

 Trainable params: 1,239 (4.84 KB)

 Non-trainable params: 17 (72.00 B)

In [34]:
from keras.layers import Input, Normalization, Dense , Concatenate
from keras.models import Model
from keras.datasets import california_housing


(X_train, y_train), (X_test, y_test) = california_housing.load_data()
X_train, y_train = X_train[:14000], y_train[:14000]
X_valid, y_valid = X_train[14000:], y_train[14000:]

input_wide = Input(shape=[5], name="input_wide") # Features from 0 -> 4
input_deep = Input(shape =[2], name="input_deep") # Features from 2 -> 7

norm_layer_wide = Normalization()
norm_layer_deep = Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = Dense(30, activation="relu")(norm_deep)
hidden2 = Dense(30,activation="relu")(hidden1)

concat = Concatenate()([norm_wide,norm_deep])

output = Dense(1, name="output")(concat)
auxilary_output = Dense(1, name="aux_output")(hidden2) # Added ana auxilary output for he deep branch

model = Model(inputs=[input_wide, input_deep], outputs = [output,auxilary_output])

In [ ]:
X_train.shape
type(X_train_wide)
type(X_train_deep)  

numpy.ndarray

In [ ]:
from keras.optimizers import Adam
import tensorflow as tf 
tf.keras.backend.clear_session()
from keras.metrics import RootMeanSquaredError

model.compile(loss = ("mse","mse"), loss_weights=(0.9, 0.1), metrics = [RootMeanSquaredError(name='rmse'), RootMeanSquaredError(name='rmse')], optimizer = Adam(learning_rate=1e-3))

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, -2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, -2:]

X_test_wide, X_test_deep = X_test[:, :5], X_test[:, -2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    {"input_wide": X_train_wide, "input_deep": X_train_deep},
    {"output": y_train, "aux_output": y_train},
    epochs=20,
    validation_data=(
        {"input_wide": X_valid_wide, "input_deep": X_valid_deep},
        {"output": y_valid, "aux_output": y_valid}
    )
)

# Evaluate
eval_results = model.evaluate(
    {"input_wide": X_test_wide, "input_deep": X_test_deep},
    {"output": y_test, "aux_output": y_test},
    return_dict=True
)


y_pred_main, y_pred_aux = model.predict({"input_wide":X_new_wide, "input_deep": X_new_deep } )



Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - aux_output_loss: 7052359680.0000 - aux_output_rmse: 83967.5547 - loss: 50920222720.0000 - output_loss: 55794290688.0000 - output_rmse: 236200.6406
Epoch 2/20
 40/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - aux_output_loss: 7580460544.0000 - aux_output_rmse: 87027.3047 - loss: 51606249472.0000 - output_loss: 56497999872.0000 - output_rmse: 237625.8125

2025-07-09 22:02:40.933645: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/deepL/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - aux_output_loss: 7050586112.0000 - aux_output_rmse: 83957.0234 - loss: 50919804928.0000 - output_loss: 55794016256.0000 - output_rmse: 236200.0625
Epoch 3/20
 53/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - aux_output_loss: 7534800896.0000 - aux_output_rmse: 86772.6172 - loss: 51696361472.0000 - output_loss: 56603205632.0000 - output_rmse: 237862.6406 

2025-07-09 22:02:42.148126: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - aux_output_loss: 7048726528.0000 - aux_output_rmse: 83945.9375 - loss: 50919399424.0000 - output_loss: 55793786880.0000 - output_rmse: 236199.5625
Epoch 4/20
 58/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - aux_output_loss: 7502083072.0000 - aux_output_rmse: 86584.5469 - loss: 51682746368.0000 - output_loss: 56591708160.0000 - output_rmse: 237842.9219

...

438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - aux_output_loss: 7024910336.0000 - aux_output_rmse: 83803.8828 - loss: 50913406976.0000 - output_loss: 55789760512.0000 - output_rmse: 236191.0469
Epoch 20/20
 81/438 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - aux_output_loss: 7361892864.0000 - aux_output_rmse: 85772.6797 - loss: 51519905792.0000 - output_loss: 56426360832.0000 - output_rmse: 237507.8750

2025-07-09 22:02:59.978146: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - aux_output_loss: 7023773184.0000 - aux_output_rmse: 83797.0938 - loss: 50913054720.0000 - output_loss: 55789514752.0000 - output_rmse: 236190.5312
 60/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - aux_output_loss: 7513225728.0000 - aux_output_rmse: 86660.9844 - loss: 51717242880.0000 - output_loss: 56628801536.0000 - output_rmse: 237890.0312

2025-07-09 22:03:00.905353: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - aux_output_loss: 7365084672.0000 - aux_output_rmse: 85805.7344 - loss: 51845464064.0000 - output_loss: 56787734528.0000 - output_rmse: 238264.6875
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


In [ ]:
aux_output_loss  = , aux_output_rmse, loss , output_loss, output_rmse = eval_results


{'aux_output_loss': 7015543808.0,
 'aux_output_rmse': 83758.84375,
 'loss': 51252068352.0,
 'output_loss': 56167247872.0,
 'output_rmse': 236996.296875}

In [45]:
import pandas as pd 
pd.DataFrame(y_train)
y_train[:5]


array([252300., 146900., 290900., 141300., 500001.], dtype=float32)